In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import sklearn.metrics as sk

from utils import get_measures, print_measures

text_features = torch.from_numpy(np.load("../MCM/features/test_labels.npy"))

np_data = np.load("../MCM/features/ImageNet_image_features.npz")
id_features = torch.from_numpy(np_data['image_features'])
id_labels = np_data['labels']

ood_features = []
for ood_dataset in ["iNaturalist",  "places365", "SUN",  "dtd"
                   ]:
    np_data = np.load(f"../MCM/features/{ood_dataset}_image_features.npz")
    ood_features.append(torch.from_numpy(np_data['image_features']))
ood_features = torch.cat(ood_features, dim=0)

id_features = id_features / id_features.norm(dim=-1, keepdim=True)
ood_features = ood_features / ood_features.norm(dim=-1, keepdim=True)
text_features = text_features / text_features.norm(dim=-1, keepdim=True)

text_features.shape, id_features.shape, ood_features.shape

(torch.Size([1000, 512]), torch.Size([50000, 512]), torch.Size([35640, 512]))

In [2]:
def cal_scores(text_features, image_features):
    output = image_features @ text_features.T
    smax = F.softmax(output / 1., dim=1).numpy()
    return np.max(smax, axis=1)

id_scores = cal_scores(text_features, id_features)
ood_scores = cal_scores(text_features, ood_features)

auroc, aupr, fpr = get_measures(id_scores, ood_scores)
print_measures(None, auroc, aupr, fpr, "")

FPR95:			41.86
AUROC: 			91.19
AUPR:  			93.43


In [8]:
import models

res = []

for i in range(1000):
    m = models.vMF(512)
    try:
        params = torch.load(f"../vmf-lib/output/vmf_model_params/{i}.pth")
        m.set_params(params[0], params[1])
        likelyhood = m.forward(id_features)
        res.append(likelyhood[:, None])
    except:
        pass
res = torch.cat(res, dim=1)
id_scores, _ = res.max(dim=1)

In [9]:
res = []
for i in range(1000):
    m = models.vMF(512)
    try:
        params = torch.load(f"../vmf-lib/output/vmf_model_params/{i}.pth")
        m.set_params(params[0], params[1])
        likelyhood = m.forward(ood_features)
        res.append(likelyhood[:, None])
    except:
        pass
res = torch.cat(res, dim=1)
ood_scores, _ = res.max(dim=1)

In [10]:
auroc, aupr, fpr = get_measures(id_scores.detach().numpy(), ood_scores.detach().numpy())
print_measures(None, auroc, aupr, fpr, "")

FPR95:			95.10
AUROC: 			73.71
AUPR:  			82.68


In [11]:
id_scores

tensor([1136.3324, 1296.3062, 1139.1367,  ..., 1061.9177, 1425.6177,
        1090.8665], grad_fn=<MaxBackward0>)